In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense



In [ ]:
df = pd.read_csv('DailyDelhiClimateTrain.csv')

In [ ]:
df = df.drop(["humidity","wind_speed", "meanpressure"], axis=1)

In [ ]:
train_size = int(df.shape[0]*0.8)

In [ ]:
train = df.iloc[:train_size, 1]
test = df.iloc[train_size:, 1]

In [ ]:
train = np.array(train).reshape(-1, 1)
test = np.array(test).reshape(-1, 1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
train_sc = scaler.fit_transform(train)
test_sc = scaler.transform(test)

In [ ]:
window_size = 10

def create_time_windows(data, window_size):
    windows = []
    labels = []
    for i in range(len(data) - window_size):
        window = data[i:i + window_size]
        windows.append(window)
        
        labels.append(data[i + window_size])  # Ostatni element z okna jako etykieta y
    return np.array(windows), np.array(labels)

X_train, y_train = create_time_windows(train_sc, window_size)
X_test, y_test = create_time_windows(test_sc, window_size)

In [ ]:
model = Sequential()
model.add(Dense(12, input_dim = window_size, activation = 'relu'))
model.add(Dense(1, activation = 'linear'))

model.summary()

model.compile(optimizer='adam',
              loss='mean_squared_error')
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
plt.plot(y_test)
plt.plot(y_pred)
plt.xlabel("Sample")
plt.ylabel("Temperature [C]")
plt.legend()

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

In [ ]:

#r2_score to metryka oceny stosowana do pomiaru dopasowania modelu do danych. Jest to również znane jako współczynnik 
#determinacji (Coefficient of Determination). r2_score mierzy, jak dobrze model regresji przewiduje zmienność zależnej 
#zmiennej względem wartości przewidywanych przez model.

#Wartość r2_score znajduje się w przedziale od 0 do 1.